In [6]:
# Step 1: Imports
import os
import subprocess
import whisper
import json
import time
import shutil
import spacy
from keybert import KeyBERT
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from pathlib import Path
from datetime import timedelta


In [7]:
# Step 2: Configurations
YOUTUBE_URL = input("Enter YouTube Video URL: ").strip()
OUTPUT_DIR = Path("E:\projects\youtube chaptering")
MP3_DIR = OUTPUT_DIR / "mp3"
TRANSCRIPT_FILE = OUTPUT_DIR / "transcript.txt"
CHAPTERS_JSON = OUTPUT_DIR / "chapters.json"
FFMPEG_PATH = "ffmpeg"  # Assuming ffmpeg is in PATH
os.makedirs(MP3_DIR, exist_ok=True)


In [15]:
# Step 3: Download and extract audio
def download_audio(youtube_url, output_dir):
    print("[INFO] Downloading and extracting audio...")
    command = [
        "yt-dlp", youtube_url,
        "-x", "--audio-format", "mp3", "--audio-quality", "192K",
        "--output", str(output_dir / "%(id)s.%(ext)s"),
        "--ffmpeg-location", FFMPEG_PATH,
        "--no-overwrites", "--continue",
        "--progress", "-N", "5"
    ]
    subprocess.run(command)

download_audio(YOUTUBE_URL, MP3_DIR)


[INFO] Downloading and extracting audio...


In [ ]:
import subprocess
from pathlib import Path

# Define your input and output paths
input_file = Path("E:/projects/youtube chaptering/mp3/ARQMEP6SH14.webm")
output_file = input_file.with_suffix(".mp3")  # same name, just .mp3 extension

# FFmpeg command
ffmpeg_cmd = [
    "ffmpeg",
    "-i", str(input_file),
    "-vn",                # no video
    "-ar", "44100",       # audio rate
    "-ac", "2",           # audio channels
    "-b:a", "192k",       # audio bitrate
    str(output_file)
]

# Run the conversion
try:
    subprocess.run(ffmpeg_cmd, check=True)
    print(f"[SUCCESS] Converted to MP3: {output_file}")
except subprocess.CalledProcessError as e:
    print(f"[ERROR] FFmpeg failed: {e}")


In [ ]:
print("Looking for MP3 files in:", MP3_DIR)


Looking for MP3 files in: E:\projects\youtube chaptering\mp3


In [ ]:
# Step 4: Transcribe with Whisper
model = whisper.load_model("medium")

def transcribe_audio(mp3_folder):
    mp3_files = list(mp3_folder.glob("*.mp3"))
    assert mp3_files, "No MP3 files found!"
    audio_path = str(mp3_files[0])
    print(f"[INFO] Transcribing {audio_path}")
    result = model.transcribe(audio_path, verbose=True)
    return result["segments"]

segments = transcribe_audio(MP3_DIR)


[INFO] Transcribing E:\projects\youtube chaptering\mp3\ARQMEP6SH14.mp3


e:\Anaconda\envs\ytch\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Hindi
[00:00.000 --> 00:01.160]  नवस्कार दोस्सों!
[00:01.160 --> 00:05.200]  चायवाला, पंचरवाला, कूडेवाला, बार डानसर और वेटर
[00:05.200 --> 00:09.000]  ये कुछ प्रोफेशन्स हैं, कुछ जॉब्स हैं जो कुछ लोग करते हैं
[00:09.000 --> 00:14.200]  लेकिन हमारे देश में अकसर इन शबदों को गालियों की तरह इस्तिमाल किया जाता हैं
[00:14.200 --> 00:17.960]  आजके इस शॉर्ट वीडियो में मैं सोशल इवल पर थोड़ी बात करना चाहुँगा
[00:17.960 --> 00:19.960]  जोकी हैं क्लास्सिसम्
[00:19.960 --> 00:22.760]  सही सुना अपने कास्टिसम् नहीं क्लास्सिसम्
[00:22.880 --> 00:25.580]  पर कास्टिसम् हो थे कि किसी एक लिए कास्ट के किलाफ भ्हेत ROBERT K. swallowed by Bία
[00:25.580 --> 00:27.040]  � steepdown regain
[00:27.040 --> 00:31.120]  हर अलगगी कास्त के लिए की ज़ुकयरी से जाइल करना पर तो make a difference
[00:31.120 --> 00:34.060]  People of númer प remotस Straße
[00:34.060 --> 00:36.660]  अलग-अलग सोशिल क्लास के लोगों के 

In [ ]:
# Step 5: Save raw transcript
with open(TRANSCRIPT_FILE, "w", encoding="utf-8") as f:
    for seg in segments:
        start = str(timedelta(seconds=int(seg['start'])))
        end = str(timedelta(seconds=int(seg['end'])))
        f.write(f"[{start} - {end}]: {seg['text'].strip()}\n")
print("[INFO] Transcript saved.")


[INFO] Transcript saved.


In [8]:
# Step 6: Load Models for Summarization and Keyword Extraction
print("[INFO] Loading mBART and KeyBERT...")
summarizer_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "hi_IN"
keybert_model = KeyBERT()
nlp = spacy.load("en_core_web_sm")


[INFO] Loading mBART and KeyBERT...


In [3]:
pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to c:\users\hp5cd\appdata\local\temp\pip-req-build-kbll841n
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\hp5cd\AppData\Local\Temp\pip-req-build-kbll841n'
